# state conversion


In [2]:
import unicodedata

def convert_to_unicode(name):
    name_unicode = unicodedata.normalize('NFD', name)
    name_unicode = ''.join(c for c in name_unicode if unicodedata.category(c) != 'Mn')
    return name_unicode

state_codes_file = "../supporting_files/state_to_code.txt"
with open(state_codes_file, "r") as f:
    state_codes = f.readlines()

state2code = {}
for line in state_codes:
    parts = line.strip().split(" ")
    code, state = parts[0], " ".join(parts[1:]).lower()
    state = convert_to_unicode(state)
    state2code[state] = code

state2code['aguascalientes']


'AGU'

In [3]:
import pandas as pd

muni_list_mx_file = "../supporting_files/CAT_ENT_MUN_POB_ENOE 062021.xlsx"
df_muni_mx = pd.read_excel(muni_list_mx_file)
df_muni_mx = df_muni_mx.iloc[:-2]
df_muni_mx.head()


,Clave Entidad,Nombre Entidad,Clave Municipio,Nombre Municipio,Población total
0,1,AGUASCALIENTES,1,AGUASCALIENTES,948990.0
1,1,AGUASCALIENTES,2,ASIENTOS,51536.0
2,1,AGUASCALIENTES,3,CALVILLO,58250.0
3,1,AGUASCALIENTES,4,COSIO,17000.0
4,1,AGUASCALIENTES,5,JESUS MARIA,129929.0


In [4]:
# Corrections
df_muni_mx["Nombre  Entidad"] = df_muni_mx["Nombre  Entidad"].apply(lambda x: "COAHUILA" if x == "COAHUILA DE ZARAGOZA".upper() else x)
df_muni_mx["Nombre  Entidad"] = df_muni_mx["Nombre  Entidad"].apply(lambda x: "MEXICO CITY" if x == "CIUDAD DE MEXICO".upper() else x)
df_muni_mx["Nombre  Entidad"] = df_muni_mx["Nombre  Entidad"].apply(lambda x: "STATE OF MEXICO" if x == 'mexico (estado de mexico)'.upper() else x)
df_muni_mx["Nombre  Entidad"] = df_muni_mx["Nombre  Entidad"].apply(lambda x: "michoacan".upper() if x == 'michoacan de ocampo'.upper() else x)
df_muni_mx["Nombre  Entidad"] = df_muni_mx["Nombre  Entidad"].apply(lambda x: "veracruz".upper() if x == 'veracruz de ignacio de la llave'.upper() else x)
df_muni_mx


,Clave Entidad,Nombre Entidad,Clave Municipio,Nombre Municipio,Población total
0,1,AGUASCALIENTES,1,AGUASCALIENTES,948990.0
1,1,AGUASCALIENTES,2,ASIENTOS,51536.0
2,1,AGUASCALIENTES,3,CALVILLO,58250.0
3,1,AGUASCALIENTES,4,COSIO,17000.0
4,1,AGUASCALIENTES,5,JESUS MARIA,129929.0
...,...,...,...,...,...
2465,32,ZACATECAS,54,VILLA HIDALGO,19446.0
2466,32,ZACATECAS,55,VILLANUEVA,31558.0
2467,32,ZACATECAS,56,ZACATECAS,149607.0
2468,32,ZACATECAS,57,TRANCOSO,20455.0


In [5]:
df_muni_mx["STATE_CODE"] = df_muni_mx["Nombre  Entidad"].apply(lambda x: state2code[x.lower()])


In [6]:
state_num_codes = df_muni_mx["Clave Entidad"].tolist()
muni_codes = df_muni_mx["Clave Municipio"].tolist()
muni_names = df_muni_mx["Nombre  Municipio"].tolist()
state_alpha_codes = df_muni_mx["STATE_CODE"].tolist()
muni_names = [convert_to_unicode(x.lower()) for x in muni_names]


In [7]:
COUNTY_MUNI_CODE = []
for i in range(len(muni_codes)):
    state_code = str(state_num_codes[i]).zfill(2)
    muni_code = str(muni_codes[i]).zfill(3)
    COUNTY_MUNI_CODE.append(f"MX{state_code}{muni_code}")

df_muni_mx["COUNTY_MUNI_CODE"] = COUNTY_MUNI_CODE
df_muni_mx


,Clave Entidad,Nombre Entidad,Clave Municipio,Nombre Municipio,Población total,STATE_CODE,COUNTY_MUNI_CODE
0,1,AGUASCALIENTES,1,AGUASCALIENTES,948990.0,AGU,MX01001
1,1,AGUASCALIENTES,2,ASIENTOS,51536.0,AGU,MX01002
2,1,AGUASCALIENTES,3,CALVILLO,58250.0,AGU,MX01003
3,1,AGUASCALIENTES,4,COSIO,17000.0,AGU,MX01004
4,1,AGUASCALIENTES,5,JESUS MARIA,129929.0,AGU,MX01005
...,...,...,...,...,...,...,...
2465,32,ZACATECAS,54,VILLA HIDALGO,19446.0,ZAC,MX32054
2466,32,ZACATECAS,55,VILLANUEVA,31558.0,ZAC,MX32055
2467,32,ZACATECAS,56,ZACATECAS,149607.0,ZAC,MX32056
2468,32,ZACATECAS,57,TRANCOSO,20455.0,ZAC,MX32057


In [8]:
st_mu_names = []
for i in range(len(muni_codes)):
    st_mu_names.append(f"{state_alpha_codes[i]}${muni_names[i]}")

df_muni_mx["ST_MUNI_NAME"] = st_mu_names
df_muni_mx


,Clave Entidad,Nombre Entidad,Clave Municipio,Nombre Municipio,Población total,STATE_CODE,COUNTY_MUNI_CODE,ST_MUNI_NAME
0,1,AGUASCALIENTES,1,AGUASCALIENTES,948990.0,AGU,MX01001,AGU$aguascalientes
1,1,AGUASCALIENTES,2,ASIENTOS,51536.0,AGU,MX01002,AGU$asientos
2,1,AGUASCALIENTES,3,CALVILLO,58250.0,AGU,MX01003,AGU$calvillo
3,1,AGUASCALIENTES,4,COSIO,17000.0,AGU,MX01004,AGU$cosio
4,1,AGUASCALIENTES,5,JESUS MARIA,129929.0,AGU,MX01005,AGU$jesus maria
...,...,...,...,...,...,...,...,...
2465,32,ZACATECAS,54,VILLA HIDALGO,19446.0,ZAC,MX32054,ZAC$villa hidalgo
2466,32,ZACATECAS,55,VILLANUEVA,31558.0,ZAC,MX32055,ZAC$villanueva
2467,32,ZACATECAS,56,ZACATECAS,149607.0,ZAC,MX32056,ZAC$zacatecas
2468,32,ZACATECAS,57,TRANCOSO,20455.0,ZAC,MX32057,ZAC$trancoso


In [9]:
df_muni_mx.to_csv("../generated_files/muni_list_mx_v2.csv", index=False)


## USA


In [10]:
usa_counties_file = "../supporting_files/united-states.xlsx"
df_usa = pd.read_excel(usa_counties_file)


In [11]:
geo_areas = df_usa['Geographic area name']
indices_states = []
for i in range(len(geo_areas)):
    if len(geo_areas[i].split(",")) < 2:
        indices_states.append(i)

df_usa = df_usa.drop(indices_states)


In [12]:
geo_areas = df_usa['Geographic area name'].tolist()
states = []
counties = []
for i in range(len(geo_areas)):
    if len(geo_areas[i].split(",")) == 2:
        states.append(geo_areas[i].split(",")[1])
        counties.append(geo_areas[i].split(",")[0])


In [13]:
df_usa["STATE"] = states
df_usa["COUNTY"] = counties


In [14]:
def convert_to_unicode(name):
    name_unicode = unicodedata.normalize('NFD', name)
    name_unicode = ''.join(c for c in name_unicode if unicodedata.category(c) != 'Mn')
    return name_unicode

def standardize_name(name):
    name = name.lower()
    name = convert_to_unicode(name)
    name = name.split(" county")[0]
    name = name.split(" city")[0]
    return name

def standardize_ccode(ccode):
    if len(ccode) == 4:
        return "US0" + ccode
    if len(ccode) == 5:
        return "US" + ccode
    return ccode


In [15]:
ccodes = df_usa['FIPS State and County Codes'].tolist()
ccodes_std = [standardize_ccode(str(x)) for x in ccodes]
df_usa["CCODE_STD"] = ccodes_std


In [16]:
states = df_usa['STATE'].tolist()
states_codes = [state2code[x.lower().lstrip()] if x.lower().lstrip() in state2code else None for x in states]
df_usa["STATE_CODE"] = states_codes


In [17]:
df_usa = df_usa.dropna()


In [18]:
county_names = df_usa['COUNTY'].tolist()
county_names_std = [standardize_name(x) for x in county_names]
df_usa["COUNTY_STD"] = county_names_std


In [19]:
state_codes_alpha = df_usa['STATE_CODE'].tolist()
county_names_std = df_usa['COUNTY_STD'].tolist()
df_usa["ST_CO"] = [f"{state_codes_alpha[i]}${county_names_std[i]}" for i in range(len(df_usa))]


In [20]:
df_usa.to_csv("../generated_files/usa_counties_v2.csv", index=False)


In [21]:
st_co2cmc = {}
state_co_mx = df_muni_mx['ST_MUNI_NAME'].tolist()
muni_codes_mx = df_muni_mx['COUNTY_MUNI_CODE'].tolist()
state_co_us = df_usa['ST_CO'].tolist()
muni_codes_us = df_usa['CCODE_STD'].tolist()

for i in range(len(state_co_mx)):
    st_co2cmc[state_co_mx[i]] = muni_codes_mx[i]

for i in range(len(state_co_us)):
    st_co2cmc[state_co_us[i]] = muni_codes_us[i]


In [22]:
# missing entries to be added
st_co2cmc["SD$oglala lakota"] = "US46102"
st_co2cmc["OAX$san juan mixtepec -dto. 26 -"] = "MX20209"
st_co2cmc["OAX$san pedro mixtepec -dto. 22 -"] = "MX20318"
st_co2cmc["VER$medellin"] = "MX30105"
st_co2cmc["OAX$santiago chazumba"] = "MX20459"

In [23]:
import json

outfile = "../supporting_files/county2municodes_ultimate_dict.json"

with open(outfile, "w") as f:
    json.dump(st_co2cmc, f, indent=4)
